In [ ]:
pip install pandas

In [2]:
import os
import json
import pandas as pd
import traceback


In [4]:
from langchain_ollama import ChatOllama


In [5]:
llm = ChatOllama(
    model="gemma3:4b",
    temperature=0.5
)



In [6]:
llm


ChatOllama(model='gemma3:4b', temperature=0.5)

In [9]:
%pip install langchain-community
%pip install PyPDF2





Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_ollama import ChatOllama

from PyPDF2 import PdfReader
import os
from langchain_core.output_parsers import StrOutputParser

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(
    model="gemma3:4b",
    temperature=0.5
)

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} briefly."
)

chain = prompt | llm | StrOutputParser()

In [9]:
llm

ChatOllama(model='gemma3:4b', temperature=0.5)

In [23]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [14]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [10]:

TEMPLATE = """
Text:
{text}

You are an expert MCQ maker.

STRICT INSTRUCTIONS (VERY IMPORTANT):
- Return ONLY valid JSON
- Do NOT include explanations
- Do NOT include markdown
- Do NOT include text outside JSON
- JSON must be directly parseable using json.loads()

The JSON format MUST be exactly:

{{
  "quiz": [
    {{
      "question": "string",
      "options": {{
        "A": "string",
        "B": "string",
        "C": "string",
        "D": "string"
      }},
      "answer": "A"
    }}
  ]
}}

Create exactly {number} questions for {subject} students in a {tone} tone.
"""


In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template=TEMPLATE,
    validate_template=True
)

In [16]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [12]:
TEMPLATE2 = """
You are an expert English grammarian and educator.

TASK:
Review the following multiple choice quiz created for {subject} students.

INSTRUCTIONS:
- Analyze the difficulty and complexity of the quiz
- Use a maximum of 50 words for the analysis
- If the quiz matches the students’ cognitive level, say so clearly
- If not, briefly suggest what should be improved
- Do NOT rewrite the full quiz unless absolutely necessary
- Do NOT use markdown or formatting

Quiz_MCQs:
{quiz}

Expert Review:
"""


In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [14]:
review_chain= quiz_evaluation_prompt| llm | StrOutputParser()

In [17]:
generate_evaluate_chain = quiz_chain | review_chain


In [18]:
file_path=r"C:\Users\Disha shetty\MCQgenerator\data.txt"

In [19]:
file_path

'C:\\Users\\Disha shetty\\MCQgenerator\\data.txt'

In [20]:
with open(file_path, 'r') as file:
    TEXT = file.read()


In [21]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[3][4]

From a theoretical viewpoint, probably approximately corr

In [24]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER=5 
SUBJECT="MACHINE LEARNING"
TONE="simple"

In [27]:
import json
import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# ---------------- LLM ----------------
llm = ChatOllama(
    model="gemma3:4b",
    temperature=0.5
)

# ---------------- JSON SAFETY ----------------
def extract_json(text: str) -> str:
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError("No JSON found in output")
    return match.group()

# ---------------- QUIZ PROMPT ----------------
quiz_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template="""
Text:
{text}

You are an expert MCQ maker.

STRICT INSTRUCTIONS:
- Return ONLY valid JSON
- Do NOT include explanations
- Do NOT include markdown
- Do NOT include text outside JSON

FORMAT:

{{
  "quiz": [
    {{
      "question": "string",
      "options": {{
        "A": "string",
        "B": "string",
        "C": "string",
        "D": "string"
      }},
      "answer": "A"
    }}
  ]
}}

Create exactly {number} MCQs for {subject} students in a {tone} tone.
"""
)

quiz_chain = quiz_prompt | llm | StrOutputParser()

# ---------------- REVIEW PROMPT ----------------
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template="""
You are an expert educator.

Review the following quiz for {subject} students.
Analyze difficulty and suggest improvements (max 50 words).

Quiz:
{quiz}
"""
)

review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

# ---------------- FINAL PIPELINE ----------------
def generate_evaluate_chain(inputs: dict) -> dict:
    # Generate quiz
    raw_quiz = quiz_chain.invoke(inputs)

    # Clean JSON
    quiz_json = extract_json(raw_quiz)

    # Review quiz
    review = review_chain.invoke(
        {
            "subject": inputs["subject"],
            "quiz": quiz_json
        }
    )

    return {
        "quiz": quiz_json,
        "review": review
    }


In [28]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE
    }
)

print(response)





{'quiz': '{\n  "quiz": [\n    {\n      "question": "What is the primary goal of machine learning?",\n      "options": {\n        "A": "To explicitly program computers to perform tasks.",\n        "B": "To allow computers to learn from data and make predictions.",\n        "C": "To solely rely on pre-defined rules for problem-solving.",\n        "D": "To eliminate the need for human input in any process."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Deep learning utilizes which type of algorithm?",\n      "options": {\n        "A": "Rule-based systems.",\n        "B": "Neural networks.",\n        "C": "Decision trees.",\n        "D": "Linear regression models."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Predictive analytics refers to:",\n      "options": {\n        "A": "The process of creating data visualizations.",\n        "B": "The application of machine learning to business problems.",\n        "C": "The study of historical data trends.",

In [30]:
print(type(response))


<class 'dict'>


In [31]:
response

{'quiz': '{\n  "quiz": [\n    {\n      "question": "What is the primary goal of machine learning?",\n      "options": {\n        "A": "To explicitly program computers to perform tasks.",\n        "B": "To allow computers to learn from data and make predictions.",\n        "C": "To solely rely on pre-defined rules for problem-solving.",\n        "D": "To eliminate the need for human input in any process."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Deep learning utilizes which type of algorithm?",\n      "options": {\n        "A": "Rule-based systems.",\n        "B": "Neural networks.",\n        "C": "Decision trees.",\n        "D": "Linear regression models."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Predictive analytics refers to:",\n      "options": {\n        "A": "The process of creating data visualizations.",\n        "B": "The application of machine learning to business problems.",\n        "C": "The study of historical data trends.",

In [32]:
quiz_response = quiz_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE
})

print(repr(quiz_response))


'```json\n{\n  "quiz": [\n    {\n      "question": "What is the primary goal of machine learning?",\n      "options": {\n        "A": "To explicitly program computers to perform tasks.",\n        "B": "To allow computers to learn from data and make predictions.",\n        "C": "To solely rely on pre-defined rules for problem-solving.",\n        "D": "To build entirely new hardware components."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Deep learning utilizes which type of algorithm?",\n      "options": {\n        "A": "Rule-based systems.",\n        "B": "Neural networks.",\n        "C": "Linear regression models.",\n        "D": "Decision trees."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Predictive analytics refers to:",\n      "options": {\n        "A": "The process of manually analyzing data without algorithms.",\n        "B": "The application of machine learning to solve business problems.",\n        "C": "The creation of physical mode

In [33]:
import json
import re

def extract_json(text: str) -> str:
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found")
    return match.group()

clean_json = extract_json(quiz_response)
quiz_data = json.loads(clean_json)

print("✅ JSON parsed successfully")



✅ JSON parsed successfully


In [34]:
print(type(quiz_data))          # dict
print(quiz_data.keys())         # should contain 'quiz'
print(type(quiz_data["quiz"])) # list


<class 'dict'>
dict_keys(['quiz'])
<class 'list'>


In [35]:
print("TYPE:", type(response))
print("REPR:", repr(response))


TYPE: <class 'dict'>
REPR: {'quiz': '{\n  "quiz": [\n    {\n      "question": "What is the primary goal of machine learning?",\n      "options": {\n        "A": "To explicitly program computers to perform tasks.",\n        "B": "To allow computers to learn from data and make predictions.",\n        "C": "To solely rely on pre-defined rules for problem-solving.",\n        "D": "To eliminate the need for human input in any process."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Deep learning utilizes which type of algorithm?",\n      "options": {\n        "A": "Rule-based systems.",\n        "B": "Neural networks.",\n        "C": "Decision trees.",\n        "D": "Linear regression models."\n      },\n      "answer": "B"\n    },\n    {\n      "question": "Predictive analytics refers to:",\n      "options": {\n        "A": "The process of creating data visualizations.",\n        "B": "The application of machine learning to business problems.",\n        "C": "The study o

In [36]:
quiz = quiz_data.get("quiz")


In [37]:

quiz_table_data = []

for q in quiz:
    mcq = q["question"]

    options = " | ".join(
        [f"{opt}: {text}" for opt, text in q["options"].items()]
    )

    correct = q["answer"]

    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct
    })


In [38]:
quiz_table_data

[{'MCQ': 'What is the primary goal of machine learning?',
  'Choices': 'A: To explicitly program computers to perform tasks. | B: To allow computers to learn from data and make predictions. | C: To solely rely on pre-defined rules for problem-solving. | D: To build entirely new hardware components.',
  'Correct': 'B'},
 {'MCQ': 'Deep learning utilizes which type of algorithm?',
  'Choices': 'A: Rule-based systems. | B: Neural networks. | C: Linear regression models. | D: Decision trees.',
  'Correct': 'B'},
 {'MCQ': 'Predictive analytics refers to:',
  'Choices': 'A: The process of manually analyzing data without algorithms. | B: The application of machine learning to solve business problems. | C: The creation of physical models to represent data. | D: The collection of raw data for future analysis.',
  'Correct': 'B'},
 {'MCQ': 'Which of the following is a foundational element of machine learning?',
  'Choices': 'A: Quantum computing. | B: Statistical analysis and mathematical optimiz

In [39]:
quiz=pd.DataFrame(quiz_table_data)

In [40]:
quiz.to_csv("machinelearning.csv",index=False)

In [41]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_22_2025_14_58_46'

In [42]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_22_2025_14_58_49'